In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.regularizers import l1, l2
from keras.optimizers import SGD, RMSprop

In [2]:
import os
import pandas as pd
import numpy as np


from keras.preprocessing.image import ImageDataGenerator


In [3]:
root = 'leaf_classif'

In [4]:
files = [file for file in os.listdir(root) if os.path.isfile(os.path.join(root, file))]

In [5]:
import random

main_dir = root  # Replace with your main directory

def label_from_folder_name(folder_name):
    if 'healthy' in folder_name:
        return 'healthy'  # Class label for healthy images
    elif 'diseased' in folder_name:
        return 'diseased'  # Class label for unhealthy images
    else:
        return None  # No specific label found
    
def custom_flow_from_directory(directory, target_size, batch_size):
    filenames = []
    labels = []

    for folder in os.listdir(directory):
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            for root, dirs, files in os.walk(folder_path):
                # Randomly shuffle the list of files
                random.shuffle(files)

                # Take the first 3 files from the shuffled list
                selected_files = files[:3]
                for file in selected_files:
                  filenames.append(os.path.join(root, file))
                  labels.append(label_from_folder_name(root))

    filenames = np.array(filenames)
    labels = np.array(labels, dtype=str)  # Ensure labels are strings

    return ImageDataGenerator(rescale=1./255).flow_from_dataframe(
        pd.DataFrame({"filename": filenames, "class": labels}),
        x_col="filename",
        y_col="class",
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary'
    )

# Use the custom function to load data
batch_size = 32
img_height, img_width = 150, 150

train_dir = os.path.join(main_dir, 'train')
test_dir = os.path.join(main_dir, 'test')
valid_dir = os.path.join(main_dir, 'valid')

train_generator = custom_flow_from_directory(train_dir, (img_height, img_width), batch_size)
test_generator = custom_flow_from_directory(test_dir, (img_height, img_width), batch_size)
valid_generator = custom_flow_from_directory(test_dir, (img_height, img_width), batch_size)


Found 66 validated image filenames belonging to 2 classes.
Found 66 validated image filenames belonging to 2 classes.
Found 66 validated image filenames belonging to 2 classes.


In [6]:
# Build a CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
# Calculate steps_per_epoch and validation_steps
steps_per_epoch_train = train_generator.samples // batch_size
steps_per_epoch_valid = valid_generator.samples // batch_size
steps_per_epoch_test = test_generator.samples // batch_size

epochs = 10
# Add 1 extra step if there are remaining samples not included in batches
if train_generator.samples % batch_size != 0:
    steps_per_epoch_train += 1
if valid_generator.samples % batch_size != 0:
    steps_per_epoch_valid += 1
if test_generator.samples % batch_size != 0:
    steps_per_epoch_test += 1

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_valid
)

Epoch 1/10


3/3 [==============================] - 84s 41s/step - loss: 0.7308 - accuracy: 0.5606 - val_loss: 0.7301 - val_accuracy: 0.5000
Epoch 2/10
3/3 [==============================] - 65s 25s/step - loss: 0.7397 - accuracy: 0.4394 - val_loss: 0.6917 - val_accuracy: 0.5606
Epoch 3/10
3/3 [==============================] - 4453s 2226s/step - loss: 0.7378 - accuracy: 0.5000 - val_loss: 0.7093 - val_accuracy: 0.5000
Epoch 4/10
3/3 [==============================] - 29s 14s/step - loss: 0.6845 - accuracy: 0.5303 - val_loss: 0.6947 - val_accuracy: 0.5152
Epoch 5/10
3/3 [==============================] - 29s 11s/step - loss: 0.6887 - accuracy: 0.5152 - val_loss: 0.6950 - val_accuracy: 0.5152
Epoch 6/10
3/3 [==============================] - 28s 10s/step - loss: 0.6798 - accuracy: 0.6970 - val_loss: 0.6951 - val_accuracy: 0.5000
Epoch 7/10
3/3 [==============================] - 17s 7s/step - loss: 0.7554 - accuracy: 0.5000 - val_loss: 0.7715 - val_accuracy: 0.5000
Epoch 8/10
3/3 [======

In [8]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, steps=steps_per_epoch_test)

3/3 [==============================] - 15s 4s/step - loss: 0.6926 - accuracy: 0.5000


Fine tune CNN model - doubling number of epochs to 20, reducing steps size by half

In [9]:
# Calculate steps_per_epoch and validation_steps
steps_per_epoch_train = train_generator.samples // (batch_size * 2)
steps_per_epoch_valid = valid_generator.samples // (batch_size * 2)
steps_per_epoch_test = test_generator.samples // (batch_size * 2)

epochs = 20
# Add 1 extra step if there are remaining samples not included in batches
if train_generator.samples % batch_size != 0:
    steps_per_epoch_train += 1
if valid_generator.samples % batch_size != 0:
    steps_per_epoch_valid += 1
if test_generator.samples % batch_size != 0:
    steps_per_epoch_test += 1

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_valid
)

Epoch 1/20
2/2 [==============================] - 28s 28s/step - loss: 0.6440 - accuracy: 0.6471 - val_loss: 0.7038 - val_accuracy: 0.5156
Epoch 2/20
2/2 [==============================] - 27s 20s/step - loss: 0.6187 - accuracy: 0.6875 - val_loss: 0.7445 - val_accuracy: 0.4844
Epoch 3/20
2/2 [==============================] - 22s 21s/step - loss: 0.6195 - accuracy: 0.7059 - val_loss: 0.7198 - val_accuracy: 0.5312
Epoch 4/20
2/2 [==============================] - 21s 14s/step - loss: 0.5787 - accuracy: 0.7353 - val_loss: 0.7408 - val_accuracy: 0.4844
Epoch 5/20
2/2 [==============================] - 27s 20s/step - loss: 0.5657 - accuracy: 0.7344 - val_loss: 0.7410 - val_accuracy: 0.5156
Epoch 6/20
2/2 [==============================] - 21s 14s/step - loss: 0.5634 - accuracy: 0.7647 - val_loss: 0.7540 - val_accuracy: 0.5625
Epoch 7/20
2/2 [==============================] - 22s 15s/step - loss: 0.4980 - accuracy: 0.7647 - val_loss: 0.8475 - val_accuracy: 0.4844
Epoch 8/20
2/2 [===========

In [10]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, steps=steps_per_epoch_test)

2/2 [==============================] - 14s 7s/step - loss: 1.9228 - accuracy: 0.4375


Use a model built from a pretrained model

In [11]:
# Use pretrained model
from keras.applications import MobileNet
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

# Define input image size expected by MobileNet
img_width, img_height = 224, 224

# Load the MobileNet model without the top classification layer
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification using sigmoid activation

# Combine base model with custom top layers
mobileModel = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
mobileModel.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


17225924/17225924 [==============================] - 1s 0us/step


In [12]:
# Train the model

# Calculate steps_per_epoch and validation_steps
steps_per_epoch_train = train_generator.samples // batch_size
steps_per_epoch_valid = valid_generator.samples // batch_size
epochs = 10
# Add 1 extra step if there are remaining samples not included in batches
if train_generator.samples % batch_size != 0:
    steps_per_epoch_train += 1
if valid_generator.samples % batch_size != 0:
    steps_per_epoch_valid += 1

model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_valid
)

Epoch 1/10
3/3 [==============================] - 37s 15s/step - loss: 0.1501 - accuracy: 0.9394 - val_loss: 2.4773 - val_accuracy: 0.3939
Epoch 2/10
3/3 [==============================] - 29s 11s/step - loss: 0.1142 - accuracy: 0.9545 - val_loss: 2.5422 - val_accuracy: 0.3636
Epoch 3/10
3/3 [==============================] - 30s 12s/step - loss: 0.1430 - accuracy: 0.9242 - val_loss: 2.8153 - val_accuracy: 0.3636
Epoch 4/10
3/3 [==============================] - 30s 11s/step - loss: 0.0563 - accuracy: 1.0000 - val_loss: 3.5544 - val_accuracy: 0.4394
Epoch 5/10
3/3 [==============================] - 29s 14s/step - loss: 0.1189 - accuracy: 0.9545 - val_loss: 3.4511 - val_accuracy: 0.3636
Epoch 6/10
3/3 [==============================] - 30s 15s/step - loss: 0.0501 - accuracy: 1.0000 - val_loss: 3.2998 - val_accuracy: 0.3939
Epoch 7/10
3/3 [==============================] - 29s 11s/step - loss: 0.0596 - accuracy: 1.0000 - val_loss: 3.2265 - val_accuracy: 0.4394
Epoch 8/10
3/3 [===========

In [13]:
# Evaluate the model on the test dataset
loss, accuracy = mobileModel.evaluate(test_generator)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

3/3 [==============================] - 15s 4s/step - loss: 0.7903 - accuracy: 0.5152
Test Loss: 0.7903
Test Accuracy: 51.52%


In [14]:
# Fine tune the pretrained model

# Calculate steps_per_epoch and validation_steps
steps_per_epoch_train = train_generator.samples // (batch_size * 2)
steps_per_epoch_valid = valid_generator.samples // (batch_size * 2)
epochs = 20
# Add 1 extra step if there are remaining samples not included in batches
if train_generator.samples % batch_size != 0:
    steps_per_epoch_train += 1
if valid_generator.samples % batch_size != 0:
    steps_per_epoch_valid += 1

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_valid
)

Epoch 1/20
2/2 [==============================] - 28s 21s/step - loss: 0.0090 - accuracy: 1.0000 - val_loss: 4.8190 - val_accuracy: 0.4062
Epoch 2/20
2/2 [==============================] - 28s 21s/step - loss: 0.0124 - accuracy: 1.0000 - val_loss: 4.8598 - val_accuracy: 0.4219
Epoch 3/20
2/2 [==============================] - 21s 14s/step - loss: 0.0077 - accuracy: 1.0000 - val_loss: 5.0912 - val_accuracy: 0.4219
Epoch 4/20
2/2 [==============================] - 21s 14s/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 5.5061 - val_accuracy: 0.4062
Epoch 5/20
2/2 [==============================] - 22s 21s/step - loss: 0.0141 - accuracy: 1.0000 - val_loss: 5.5496 - val_accuracy: 0.4062
Epoch 6/20
2/2 [==============================] - 22s 15s/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 5.6877 - val_accuracy: 0.4375
Epoch 7/20
2/2 [==============================] - 22s 21s/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 6.1183 - val_accuracy: 0.3906
Epoch 8/20
2/2 [===========

In [15]:
# Evaluate the model on the test dataset
loss, accuracy = mobileModel.evaluate(test_generator)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

3/3 [==============================] - 14s 4s/step - loss: 0.7903 - accuracy: 0.5152
Test Loss: 0.7903
Test Accuracy: 51.52%


In [16]:
import os
from keras.preprocessing import image

images_to_predict_dir = "predict"

def predict_images_in_folder(model, folder_path):
    predictions = []
    image_paths = []

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(('.JPG')):
                img_path = os.path.join(root, file)
                image_paths.append(img_path)
                prediction = predict_image(model, img_path)
                predictions.append(prediction)
    return image_paths, predictions

# Function to predict a single image
def predict_image(model, img_path):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.

    prediction = model.predict(img_array)
    if prediction < 0.5:
        return "Healthy"
    else:
        return "Unhealthy"

image_paths, predictions = predict_images_in_folder(model, images_to_predict_dir)

In [17]:
import matplotlib.pyplot as plt

num_images = len(image_paths)
num_cols = 5
num_rows = -(-num_images // num_cols)  # Ceiling division

plt.figure(figsize=(15, 3 * num_rows))
for i, (img_path, prediction) in enumerate(zip(image_paths, predictions)):
    plt.subplot(num_rows, num_cols, i + 1)
    img = image.load_img(img_path, target_size=(img_height, img_width))
    plt.imshow(img)
    plt.title(f"{os.path.basename(img_path)}\nPrediction: {prediction}")
    plt.axis('off')

plt.tight_layout()
plt.show()

<Figure size 1500x0 with 0 Axes>